In [1]:
cd C:\Users\emartezorrilla\OneDrive - University of Florida\EED_PHD_EM_shared_2020\Papers_WorkinPorgress\98 Robot_FIVX-2025

C:\Users\emartezorrilla\OneDrive - University of Florida\EED_PHD_EM_shared_2020\Papers_WorkinPorgress\98 Robot_FIVX-2025


In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 2042-1F2B

 Directory of C:\Users\emartezorrilla\OneDrive - University of Florida\EED_PHD_EM_shared_2020\Papers_WorkinPorgress\98 Robot_FIVX-2025

09/28/2025  09:17 PM    <DIR>          .
09/30/2025  02:28 PM    <DIR>          ..
09/15/2025  02:06 PM    <DIR>          .ipynb_checkpoints
09/18/2025  07:42 PM    <DIR>          backup
09/19/2025  01:11 PM            76,964 Cahnging_Polarity.docx
09/27/2025  08:11 PM  (11,694,915,584) edwin_Mpi_container.tar
09/28/2025  09:18 PM    <DIR>          images_edwin
09/07/2025  09:58 AM         8,781,927 Instrucciones RRC Lite.docx
09/15/2025  01:53 PM             6,394 map_creator.ipynb
09/15/2025  01:56 PM    <DIR>          maps
09/15/2025  01:43 PM           148,846 maze_01.png
09/15/2025  01:50 PM            31,545 maze_01_binary.png
09/23/2025  09:28 AM    <DIR>          Turning_references
10/05/2025  10:38 AM    <DIR>          WROS_RD_ROS
               6 File(s) 11,703,961,260 bytes
  

In [3]:
!pip install opencv-python

Converting the image
First of all, we need to make sure the image is in binary format. Here is a Python file, credits to automaticaddison.com; we place it in utils folder that is located in the src folder of our workspace, outside navigation_bot_06 folder. This way any project (like futire navigation_bot_XXX) can use it:

convert_png_to_binary.py:

In [10]:
                    import cv2 # Import OpenCV

                    # read the image file
                    img = cv2.imread('patio.jpg')
                    
                    ret, bw_img = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY)
                    
                    # converting to its binary form
                    bw = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY)
                    
                    cv2.imwrite("patio_binary.jpg", bw_img)

True

Generating the map
In utils folder, create convert_png_to_map.py file

In [11]:
import numpy as np
import cv2
import math
import os.path

# The name of your floor plan you want to convert to a ROS map: 
strSrcFileName = "patio_binary.png"
image = cv2.imread(strSrcFileName)

strDstFileName = "patio_01"
strDstLocation = "maps/"

# We scale the map, so it is 40 meters wide
img_width_meters = 6.00

width, height, _ = image.shape

x1 = [0,width,  0,0]
y1 = [0,0,      0,height]

deltax = img_width_meters
dx = math.sqrt((x1[1]-x1[0])**2 + (y1[1]-y1[0])**2)*.05
sx = deltax / dx

deltay = img_width_meters * float(height) / width
dy = math.sqrt((x1[3]-x1[2])**2 + (y1[3]-y1[2])**2)*.05
sy = deltay/dy 

res = cv2.resize(image, None, fx=sx, fy=sy, interpolation = cv2.INTER_CUBIC)

# Convert to grey
res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
#cv2.imwrite("KEC_BuildingCorrected.pgm", res );
#      cv2.imshow("Image2", res)

completeFileNameMap = os.path.join(strDstLocation, strDstFileName +".pgm")
completeFileNameYaml = os.path.join(strDstLocation, strDstFileName +".yaml")
    
cv2.imwrite(completeFileNameMap, res )

yaml = open(completeFileNameYaml, "w")

yaml.write("image: " + strDstFileName + ".pgm\n")
yaml.write("resolution: 0.050000\n")
yaml.write("origin: [" + str(-1) + "," +  str(-1) + ", 0.000000]\n")
yaml.write("negate: 0\noccupied_thresh: 0.65\nfree_thresh: 0.196")
yaml.close()